In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [23]:
customers = pd.read_csv("Customers.csv")
transactions = pd.read_csv("Transactions.csv")
products = pd.read_csv("Products.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Customers.csv'

In [3]:
customer_transactions = pd.merge(transactions, customers, on='CustomerID', how='left')
customer_transactions = pd.merge(customer_transactions, products, on='ProductID', how='left')

In [4]:
customer_summary = customer_transactions.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    transaction_count=('TransactionID', 'count'),
    average_transaction_value=('TotalValue', 'mean')
).reset_index()

In [5]:
customer_summary = pd.merge(customer_summary, customers[['CustomerID', 'Region', 'SignupDate']], on='CustomerID', how='left')

In [6]:
customer_summary['SignupDate'] = pd.to_datetime(customer_summary['SignupDate'])
customer_summary['days_since_signup'] = (datetime.now() - customer_summary['SignupDate']).dt.days

In [7]:
scaler = StandardScaler()
features = ['total_spent', 'transaction_count', 'average_transaction_value', 'days_since_signup']
customer_summary[features] = scaler.fit_transform(customer_summary[features])

In [8]:
scaler = StandardScaler()
customer_summary[['total_spent', 'transaction_count', 'average_transaction_value', 'days_since_signup']] = \
    scaler.fit_transform(customer_summary[['total_spent', 'transaction_count', 'average_transaction_value', 'days_since_signup']])

In [9]:
customer_pairs = []

for i in range(len(customer_summary)):
    for j in range(i + 1, len(customer_summary)):  # Pair each customer with every other customer
        customer_pairs.append((customer_summary.iloc[i], customer_summary.iloc[j]))

In [10]:
pairs_data = []
for pair in customer_pairs:
    customer1, customer2 = pair
    similarity_score = cosine_similarity([customer1[features].values], [customer2[features].values])[0][0]
    pairs_data.append({
        'CustomerID_1': customer1['CustomerID'],
        'CustomerID_2': customer2['CustomerID'],
        'TotalSpent_1': customer1['total_spent'],
        'TotalSpent_2': customer2['total_spent'],
        'TransactionCount_1': customer1['transaction_count'],
        'TransactionCount_2': customer2['transaction_count'],
        'AvgTransactionValue_1': customer1['average_transaction_value'],
        'AvgTransactionValue_2': customer2['average_transaction_value'],
        'DaysSinceSignup_1': customer1['days_since_signup'],
        'DaysSinceSignup_2': customer2['days_since_signup'],
        'SimilarityScore': similarity_score
    })

In [11]:
pairs_df = pd.DataFrame(pairs_data)

In [12]:
X = pairs_df[['TotalSpent_1', 'TotalSpent_2', 'TransactionCount_1', 'TransactionCount_2', 
              'AvgTransactionValue_1', 'AvgTransactionValue_2', 'DaysSinceSignup_1', 'DaysSinceSignup_2']]
y = pairs_df['SimilarityScore']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [15]:
y_pred = model.predict(X_test)

In [16]:
customer_summary= customer_summary.head(20)

lookalike_recommendations = {}

for idx, customer_id in enumerate(customer_summary['CustomerID']):
    similarities = []
    
    # Get the feature vector for the current customer
    customer1 = customer_summary.loc[customer_summary['CustomerID'] == customer_id].iloc[0]
    customer1_features = np.array([
        customer1['total_spent'],
        customer1['transaction_count'],
        customer1['average_transaction_value'],
        customer1['days_since_signup']
    ]).reshape(1, -1)

    # Loop over the rest of the first 20 customers
    for jdx, other_customer_id in enumerate(customer_summary['CustomerID']):
        if customer_id != other_customer_id:
            # Get the feature vector for the other customer
            customer2 = customer_summary.loc[customer_summary['CustomerID'] == other_customer_id].iloc[0]
            customer2_features = np.array([
                customer2['total_spent'],
                customer2['transaction_count'],
                customer2['average_transaction_value'],
                customer2['days_since_signup']
            ]).reshape(1, -1)
            
            pair_features = np.concatenate([customer1_features, customer2_features], axis=1)
            
            predicted_similarity = model.predict(pair_features)[0]
            similarities.append((other_customer_id, predicted_similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_3_similar = similarities[:3]
    
    # Store the recommendations
    lookalike_recommendations[customer_id] = top_3_similar

/Users/anirudhtyagi/Projects/DL/venv1/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/anirudhtyagi/Projects/DL/venv1/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/anirudhtyagi/Projects/DL/venv1/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/anirudhtyagi/Projects/DL/venv1/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/anirudhtyagi/Projects/DL/venv1/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarn

In [19]:
lookalike_list = []
for customer_id, recommendations in lookalike_recommendations.items():
    # Get only top 3 recommendations for each customer
    top_3_recommendations = recommendations[:3]  # Slice to get first 3 items
    for idx, (lookalike_id, score) in enumerate(top_3_recommendations):
        lookalike_list.append([customer_id, idx+1, lookalike_id, score])

lookalike_df = pd.DataFrame(lookalike_list, columns=['CustomerID', 'Rank', 'LookalikeCustomerID', 'SimilarityScore'])

lookalike_df.to_csv('Lookalike.csv', index=False)